In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import time
import random
from urllib.parse import urlparse, parse_qs
from transformers import pipeline
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from pykakasi import kakasi  # ローマ字変換用
from matplotlib import font_manager, rcParams
import matplotlib.dates as mdates

# 必要なライブラリがインストールされていることを確認してください
# 以下のコマンドでインストールできます
# pip install transformers pandas matplotlib requests beautifulsoup4 python-dotenv pykakasi

# pykakasiの設定
kks = kakasi()
kks.setMode("J", "a")  # 漢字からローマ字への変換
conv = kks.getConverter()

# .envファイルを読み込む（非公開モデルを使用する場合はアクセストークンが必要）
load_dotenv()

# 環境変数からアクセストークンを取得（非公開モデルを使用する場合のみ）
access_token = os.getenv('HF_ACCESS_TOKEN')

# 感情分析パイプラインの初期化
# 使用しているモデルが公開モデルの場合、use_auth_tokenは不要です。
# 非公開モデルを使用する場合のみ、以下の行のコメントを外してください。
sentiment_analyzer = pipeline(
    'sentiment-analysis',
    model='koheiduck/bert-japanese-finetuned-sentiment',  # 使用するモデルを指定
    # use_auth_token=access_token  # 非公開モデルの場合はアクセストークンを設定
)

# 解析対象のメンバー名を直接指定
member_name_input = "村山 美羽"  # 解析したいメンバーの名前をここに入力

# メンバー名をローマ字に変換
member_name_en = conv.do(member_name_input).replace(" ", "_")  # スペースをアンダースコアに置換
output_filename = f"{member_name_en}_EmotionAnalysis.txt"

# 日本語フォントの設定（フォントパスは環境に応じて変更してください）
try:
    if os.name == 'nt':
        font_path = 'C:\\Windows\\Fonts\\msgothic.ttc'  # Windowsの場合
    else:
        font_path = '/usr/share/fonts/truetype/ipafont/ipagp.ttf'  # Linuxの場合
    if os.path.exists(font_path):
        font_manager.fontManager.addfont(font_path)
        rcParams['font.family'] = 'IPAGothic' if 'ipagp.ttf' in font_path else 'MS Gothic'
    else:
        print(f"指定されたフォントパスが存在しません: {font_path}")
except Exception as e:
    print(f"フォントの設定に失敗しました: {e}")
    print("日本語フォントが正しく表示されない可能性があります。")

# メンバー名とそのブログトップページのURLを取得する関数
def get_member_url(base_url, target_member_name):
    headers = {'User-Agent': 'Mozilla/5.0'}
    try:
        response = requests.get(base_url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"メンバーページの取得に失敗しました: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    members = soup.select('ul.com-blog-circle li a')

    for member in members:
        name_tag = member.select_one('p.name')
        if not name_tag:
            continue
        member_name = name_tag.get_text().strip()
        if member_name == target_member_name:
            member_url = f"https://sakurazaka46.com{member['href']}"
            return member_url

    return None

# 各ブログページをスクレイピングしてテキストを解析する関数
def scrape_blog_page(blog_url, file, time_series_data):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(blog_url, headers=headers, timeout=10)
        response.raise_for_status()

        bs4_blog = BeautifulSoup(response.text, 'html.parser')
        article = bs4_blog.select_one('article.post')

        if article:
            # 日付の取得
            year_element = article.find('span', {'class': 'ym-year'})
            month_element = article.find('span', {'class': 'ym-month'})
            day_element = article.find('p', {'class': 'date wf-a'})

            if year_element and month_element and day_element:
                year = year_element.get_text().strip()
                month = month_element.get_text().strip().zfill(2)
                day = day_element.get_text().strip().zfill(2)
                date_str = f"{year}/{month}/{day}"
            else:
                print(f"日付が見つかりませんでした: {blog_url}")
                date_str = "unknown_date"

            # 日付の解析
            try:
                date_obj = datetime.strptime(date_str, '%Y/%m/%d')
            except ValueError:
                print(f"日付の形式が不明です: {date_str}（ブログURL: {blog_url}）")
                return

            # ブログ本文のテキストを取得
            content_div = article.select_one('div.box-article')
            if content_div:
                # スクリプトやスタイル要素を削除
                for script_or_style in content_div(['script', 'style']):
                    script_or_style.decompose()

                content_text = content_div.get_text(separator="\n", strip=True)

                # 日付を書き込む
                file.write(f"\n日付: {date_str}\n")

                # テキストを行ごとに分割して解析
                lines = content_text.splitlines()
                positive_count = 0  # ポジティブな行の数
                negative_count = 0  # ネガティブな行の数

                for line in lines:
                    line = line.strip()
                    if not line:
                        continue  # 空行はスキップ

                    # 感情分析
                    try:
                        result = sentiment_analyzer(line)[0]
                        label = result['label']
                        score = result['score']
                    except Exception as e:
                        print(f"感情分析中にエラーが発生しました: {e}")
                        label = "unknown"
                        score = 0.0

                    # 行とスコアをファイルに書き込む
                    file.write(f"{line} ... スコア: {score:.2f}, ラベル: {label}\n")

                    # スコアを累積
                    if label.lower() == 'positive':
                        positive_count += 1
                    elif label.lower() == 'negative':
                        negative_count += 1

                # 時系列データに追加
                if date_obj in time_series_data:
                    time_series_data[date_obj]['positive_count'] += positive_count
                    time_series_data[date_obj]['negative_count'] += negative_count
                else:
                    time_series_data[date_obj] = {
                        'positive_count': positive_count,
                        'negative_count': negative_count
                    }

            else:
                print(f"本文が見つかりませんでした: {blog_url}")
        else:
            print(f"記事が見つかりませんでした: {blog_url}")

    except requests.RequestException as e:
        print(f"ブログページの取得に失敗しました: {e}")
    except Exception as e:
        print(f"エラーが発生しました: {e}")

# メンバーの全ブログをスクレイピングして解析する関数
def scrape_all_blogs(member_name, member_url, max_pages=100):
    # メンバーのct値とima値を取得
    parsed_url = urlparse(member_url)
    query_params = parse_qs(parsed_url.query)
    ct_value = query_params.get('ct', [''])[0]
    ima_value = query_params.get('ima', ['0000'])[0]

    # 保存先ディレクトリの設定
    base_save_dir = os.path.join('data', member_name)
    save_dir = os.path.join(base_save_dir, f"{member_name}_感情分析結果")
    os.makedirs(save_dir, exist_ok=True)

    # 出力ファイルのパス
    output_path = os.path.join(save_dir, output_filename)

    with open(output_path, 'w', encoding='utf-8') as file:
        time_series_data = {}
        current_page = 1  # ページ番号を1から開始

        # ページ1をスクレイピング（member_url）
        print(f"ページ {current_page} をスクレイピング中: {member_url}")
        headers = {'User-Agent': 'Mozilla/5.0'}
        try:
            response = requests.get(member_url, headers=headers, timeout=10)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"ページの取得に失敗しました: {e}")
            return time_series_data

        soup = BeautifulSoup(response.text, 'html.parser')

        # ブログ記事の URL を取得
        articles = soup.select('ul.com-blog-part li.box a')
        blog_links = [f"https://sakurazaka46.com{a['href']}" for a in articles if a.get('href')]

        if not blog_links:
            print(f"ブログ記事が見つかりませんでした: {member_url}")
        else:
            print(f"取得したブログ記事数: {len(blog_links)}")
            for blog_link in blog_links:
                print(f"ブログをスクレイピング中: {blog_link}")
                scrape_blog_page(blog_link, file, time_series_data)
                time.sleep(random.uniform(2, 5))  # サーバーへの負荷を軽減

        # 次のページ以降をスクレイピング
        for current_page in range(2, max_pages + 1):
            # ページURLを構築
            page_param = current_page - 1  # pageパラメータは0始まりなので、1を引く
            page_url = f"https://sakurazaka46.com/s/s46/diary/blog/list?ima={ima_value}&page={page_param}&ct={ct_value}"
            print(f"ページ {current_page} をスクレイピング中: {page_url}")

            try:
                response = requests.get(page_url, headers=headers, timeout=10)
                response.raise_for_status()
            except requests.RequestException as e:
                print(f"ページの取得に失敗しました: {e}")
                break

            soup = BeautifulSoup(response.text, 'html.parser')

            # ブログ記事の URL を取得
            articles = soup.select('ul.com-blog-part li.box a')
            blog_links = [f"https://sakurazaka46.com{a['href']}" for a in articles if a.get('href')]

            if not blog_links:
                print(f"ブログ記事が見つかりませんでした: {page_url}")
                break

            print(f"取得したブログ記事数: {len(blog_links)}")
            for blog_link in blog_links:
                print(f"ブログをスクレイピング中: {blog_link}")
                scrape_blog_page(blog_link, file, time_series_data)
                time.sleep(random.uniform(2, 5))  # サーバーへの負荷を軽減

            time.sleep(random.uniform(2, 5))  # サーバーへの負荷を軽減

    return time_series_data

# メイン処理を関数にまとめる
def main():
    base_url = 'https://sakurazaka46.com/s/s46/diary/blog/list?ima=0000'
    member_url = get_member_url(base_url, member_name_input)

    if not member_url:
        print(f"指定されたメンバー名 '{member_name_input}' が見つかりませんでした。")
        return

    print(f"メンバーのブログを解析開始: {member_name_input}")
    time_series_data = scrape_all_blogs(member_name_input, member_url)

    if not time_series_data:
        print("解析対象のデータが見つかりませんでした。")
        return

    # 時系列データをデータフレームに変換
    df = pd.DataFrame([
        {
            'date': date,
            'positive_count': counts['positive_count'],
            'negative_count': counts['negative_count']
        }
        for date, counts in time_series_data.items()
    ])
    df.sort_values('date', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # ポジティブとネガティブの割合を計算
    df['total_count'] = df['positive_count'] + df['negative_count']
    df['positive_percentage'] = (df['positive_count'] / df['total_count']) * 100
    df['negative_percentage'] = (df['negative_count'] / df['total_count']) * 100

    # テキストファイルに総スコアを記載
    base_save_dir = os.path.join('data', member_name_input)
    save_dir = os.path.join(base_save_dir, f"{member_name_input}_感情分析結果")
    with open(os.path.join(save_dir, output_filename), 'a', encoding='utf-8') as file:
        total_positive_count = df['positive_count'].sum()
        total_negative_count = df['negative_count'].sum()
        total_count = total_positive_count + total_negative_count

        file.write("\n=== 総行数 ===\n")
        file.write(f"ポジティブ行数合計: {total_positive_count}\n")
        file.write(f"ネガティブ行数合計: {total_negative_count}\n")
        if total_count > 0:
            positive_ratio = (total_positive_count / total_count) * 100
            negative_ratio = (total_negative_count / total_count) * 100
            file.write(f"ポジティブ割合（行数ベース）: {positive_ratio:.2f}%\n")
            file.write(f"ネガティブ割合（行数ベース）: {negative_ratio:.2f}%\n")
        else:
            file.write("行数の合計が0のため、割合を計算できません。\n")

    # ポジティブ・ネガティブの割合の時系列グラフを作成
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['positive_percentage'], label='Positive Percentage', marker='o')
    plt.plot(df['date'], df['negative_percentage'], label='Negative Percentage', marker='o')

    # プロット点の上に日付と割合を表示
    for i, row in df.iterrows():
        date_str = row['date'].strftime('%Y-%m-%d')
        plt.annotate(f"{date_str}\n{row['positive_percentage']:.1f}%", (row['date'], row['positive_percentage']),
                     textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
        plt.annotate(f"{date_str}\n{row['negative_percentage']:.1f}%", (row['date'], row['negative_percentage']),
                     textcoords="offset points", xytext=(0,-25), ha='center', fontsize=8)

    # X軸のラベルを調整
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    plt.xlabel('Date')
    plt.ylabel('Percentage (%)')
    plt.title(f"{member_name_input}のポジティブ・ネガティブ割合時系列グラフ")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)

    # グラフを保存
    percentage_time_series_graph_filename = os.path.join(save_dir, f"{member_name_input}_ポジティブ・ネガティブ割合時系列グラフ.png")
    plt.savefig(percentage_time_series_graph_filename, dpi=300, bbox_inches='tight')
    print(f"グラフを保存しました: {percentage_time_series_graph_filename}")
    plt.close()

    # **追加: ポジティブ・ネガティブ行数の時系列グラフを作成**
    plt.figure(figsize=(12, 6))
    plt.plot(df['date'], df['positive_count'], label='Positive Count', marker='o')
    plt.plot(df['date'], df['negative_count'], label='Negative Count', marker='o')

    # プロット点の上に日付と行数を表示
    for i, row in df.iterrows():
        date_str = row['date'].strftime('%Y-%m-%d')
        plt.annotate(f"{date_str}\n{int(row['positive_count'])}", (row['date'], row['positive_count']),
                     textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)
        plt.annotate(f"{date_str}\n{int(row['negative_count'])}", (row['date'], row['negative_count']),
                     textcoords="offset points", xytext=(0,-25), ha='center', fontsize=8)

    # X軸のラベルを調整
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.title(f"{member_name_input}のポジティブ・ネガティブ行数時系列グラフ")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.grid(True)

    # グラフを保存
    count_time_series_graph_filename = os.path.join(save_dir, f"{member_name_input}_ポジティブ・ネガティブ行数時系列グラフ.png")
    plt.savefig(count_time_series_graph_filename, dpi=300, bbox_inches='tight')
    print(f"グラフを保存しました: {count_time_series_graph_filename}")
    plt.close()

    # グラフを表示
    plt.show()

# 実行
if __name__ == "__main__":
    main()


C:\Users\n-nakagawa_d1\AppData\Local\Temp\ipykernel_568\1712262231.py:22: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kks.setMode("J", "a")  # 漢字からローマ字への変換
C:\Users\n-nakagawa_d1\AppData\Local\Temp\ipykernel_568\1712262231.py:23: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conv = kks.getConverter()
C:\Users\n-nakagawa_d1\AppData\Local\Temp\ipykernel_568\1712262231.py:44: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  member_name_en = conv.do(member_name_input).replace(" ", "_")  # スペースをアンダースコアに置換


メンバーのブログを解析開始: 遠藤 理子
ページ 1 をスクレイピング中: https://sakurazaka46.com/s/s46/diary/blog/list?ima=0000&ct=60
取得したブログ記事数: 12
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/58068?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57860?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57837?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57805?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57770?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57735?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57692?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57654?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57549?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57475?ima=0000&cd=blog
ブログをスクレイピング中: https://sakurazaka46.com/s/s46/diary/detail/57181?ima=0000&cd